## [API](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)

## SparkSession

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Apache Spark test') \
    .config("spark.sql.shuffle.partitions", "500") \
    .getOrCreate()

In [2]:
spark

## Data 
[download data](https://github.com/databricks/Spark-The-Definitive-Guide)

In [46]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

df = spark.read.format('csv') \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Users/sg0218817/Downloads/Spark-The-Definitive-Guide-master/data/retail-data/all/*.csv") \
    .coalesce(4) \
    .withColumn("date", to_date(col('InvoiceDate'), 'MM/d/yyyy H:mm')) \
    .cache()

In [4]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [45]:
df.show(2, False)

+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |date      |
+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+----------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|2010-12-01|
|536365   |71053    |WHITE METAL LANTERN               |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
+---------+---------+----------------------------------+--------+--------------+---------+----------+--------------+----------+
only showing top 2 rows



## basic aggregations

Spark SQL provides built-in methods for the most common aggregations such as count(), countDistinct(), avg(), max(), min(), etc. in the pyspark.sql.functions module. These methods are not the same as the built-in methods in the Python Standard Library, where we can find min() for example as well, hence you need to be careful not to use them interchangeably.

In many cases, there are multiple ways to express the same aggregations. For example, if we would like to compute one type of aggregate for one or more columns of the DataFrame we can just simply chain the aggregate method after a groupBy(). If we would like to use different functions on different columns, agg()comes in handy. For example agg({"salary": "avg", "age": "max"}) computes the average salary and maximum age.

In [14]:
df.select(count('InvoiceNo')) \
    .show()

+----------------+
|count(InvoiceNo)|
+----------------+
|          541909|
+----------------+



In [15]:
df.count()

541909

In [16]:
df.select(countDistinct('InvoiceNo')) \
    .show()

+-------------------------+
|count(DISTINCT InvoiceNo)|
+-------------------------+
|                    25900|
+-------------------------+



In [18]:
df.select(approx_count_distinct('InvoiceNo')) \
    .show()

+--------------------------------+
|approx_count_distinct(InvoiceNo)|
+--------------------------------+
|                           26470|
+--------------------------------+



In [19]:
df.select(first('StockCode'), last('StockCode')) \
    .show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [20]:
df.select(min('Quantity'), max('Quantity')) \
    .show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [27]:
df.select(sum('Quantity'), 
          sumDistinct('Quantity'), 
          avg('Quantity'), 
          var_pop('Quantity'), 
          var_samp('Quantity'),
          stddev_pop('Quantity'), 
          stddev_samp('Quantity')) \
    .show()

+-------------+----------------------+----------------+------------------+------------------+--------------------+---------------------+
|sum(Quantity)|sum(DISTINCT Quantity)|   avg(Quantity)| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-------------+----------------------+----------------+------------------+------------------+--------------------+---------------------+
|      5176450|                 29310|9.55224954743324|47559.303646608976| 47559.39140929867|   218.0809566344778|   218.08115785023398|
+-------------+----------------------+----------------+------------------+------------------+--------------------+---------------------+



## collect to complex types

In [28]:
df.agg(collect_set('Country'), collect_list('Country')) \
    .show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



## grouping

In [37]:
df.where('InvoiceNo = 536762') \
    .groupBy('InvoiceNo', 'CustomerId') \
    .count() \
    .show(5, False)

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|536762   |16186     |20   |
+---------+----------+-----+



In [43]:
df.where('InvoiceNo = 536762') \
    .groupBy('InvoiceNo', 'CustomerId') \
    .agg(count('Quantity').alias('quantity'), expr('count(Quantity)'), first('Country')) \
    .show(5, False)

+---------+----------+--------+---------------+---------------------+
|InvoiceNo|CustomerId|quantity|count(Quantity)|first(Country, false)|
+---------+----------+--------+---------------+---------------------+
|536762   |16186     |20      |20             |United Kingdom       |
+---------+----------+--------+---------------+---------------------+



## window functions

Window functions are a way of combining the values of ranges of rows in a DataFrame. When defining the window we can choose how to sort and group (with the partitionBy method) the rows and how wide of a window we'd like to use (described by rangeBetween or rowsBetween).

In [98]:
window_specification = Window.partitionBy('CustomerId', 'date') \
    .orderBy(desc('Quantity')) \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [99]:
max_purchase_quantity = max(col('Quantity')).over(window_specification)
purchase_dense_rank = dense_rank().over(window_specification)
purchase_rank = rank().over(window_specification)

In [101]:
df.select(
        col('InvoiceDate'), col('InvoiceNo'), col('Description'), col('Quantity'), 
        purchase_rank.alias('quantityRank'),
        purchase_dense_rank.alias('quantityDenseRank'),
        max_purchase_quantity.alias('maxQuantity')) \
    .where('CustomerId = 12347') \
    .show(100)

+---------------+---------+--------------------+--------+------------+-----------------+-----------+
|    InvoiceDate|InvoiceNo|         Description|Quantity|quantityRank|quantityDenseRank|maxQuantity|
+---------------+---------+--------------------+--------+------------+-----------------+-----------+
| 4/7/2011 10:43|   549222|ICE CREAM SUNDAE ...|     240|           1|                1|        240|
| 4/7/2011 10:43|   549222|MINI PAINT SET VI...|      36|           2|                2|        240|
| 4/7/2011 10:43|   549222|PACK OF 60 DINOSA...|      24|           3|                3|        240|
| 4/7/2011 10:43|   549222|SMALL FOLDING SCI...|      24|           3|                3|        240|
| 4/7/2011 10:43|   549222|VINTAGE HEADS AND...|      12|           5|                4|        240|
| 4/7/2011 10:43|   549222|RED DRAWER KNOB A...|      12|           5|                4|        240|
| 4/7/2011 10:43|   549222|LARGE HEART MEASU...|      12|           5|                4|   

## rollups

In [116]:
df.rollup('date', 'Country') \
    .agg(sum('Quantity').alias('sum')) \
    .orderBy(desc('sum')) \
    .where('date >= "2010-12-01" and date <= "2010-12-02" or date is null') \
    .show()

+----------+--------------+-------+
|      date|       Country|    sum|
+----------+--------------+-------+
|      null|          null|5176450|
|2010-12-01|          null|  26814|
|2010-12-01|United Kingdom|  23949|
|2010-12-02|          null|  21023|
|2010-12-02|United Kingdom|  20873|
|2010-12-01|        Norway|   1852|
|2010-12-01|        France|    449|
|2010-12-01|          EIRE|    243|
|2010-12-02|       Germany|    146|
|2010-12-01|       Germany|    117|
|2010-12-01|     Australia|    107|
|2010-12-01|   Netherlands|     97|
|2010-12-02|          EIRE|      4|
+----------+--------------+-------+



## cubes

In [119]:
df.cube('date', 'Country') \
    .agg(sum('Quantity').alias('sum')) \
    .orderBy(desc('sum')) \
    .where('date >= "2010-12-01" and date <= "2010-12-02" or date is null') \
    .show(20)

+----------+---------------+-------+
|      date|        Country|    sum|
+----------+---------------+-------+
|      null|           null|5176450|
|      null| United Kingdom|4263829|
|      null|    Netherlands| 200128|
|      null|           EIRE| 142637|
|      null|        Germany| 117448|
|      null|         France| 110480|
|      null|      Australia|  83653|
|      null|         Sweden|  35637|
|      null|    Switzerland|  30325|
|      null|          Spain|  26824|
|2010-12-01|           null|  26814|
|      null|          Japan|  25218|
|2010-12-01| United Kingdom|  23949|
|      null|        Belgium|  23152|
|2010-12-02|           null|  21023|
|2010-12-02| United Kingdom|  20873|
|      null|         Norway|  19247|
|      null|       Portugal|  16180|
|      null|        Finland|  10666|
|      null|Channel Islands|   9479|
+----------+---------------+-------+
only showing top 20 rows



## pivots

In [127]:
df.groupBy('date') \
    .pivot('Country') \
    .sum() \
    .where('date > "2011-12-05"') \
    .select('date', '`USA_sum(CAST(Quantity AS BIGINT))`', '`France_sum(CAST(Quantity AS BIGINT))`') \
    .show()

+----------+---------------------------------+------------------------------------+
|      date|USA_sum(CAST(Quantity AS BIGINT))|France_sum(CAST(Quantity AS BIGINT))|
+----------+---------------------------------+------------------------------------+
|2011-12-09|                             null|                                 105|
|2011-12-06|                             null|                                 787|
|2011-12-08|                             -196|                                  18|
|2011-12-07|                             null|                                 561|
+----------+---------------------------------+------------------------------------+



In [138]:
df.where('date between "2011-12-06" and "2011-12-09"') \
    .where('Country = "France" or Country = "USA"') \
    .groupBy('date', 'Country') \
    .sum() \
    .select('date', 'Country', 'sum(Quantity)') \
    .show()

+----------+-------+-------------+
|      date|Country|sum(Quantity)|
+----------+-------+-------------+
|2011-12-07| France|          561|
|2011-12-08|    USA|         -196|
|2011-12-09| France|          105|
|2011-12-06| France|          787|
|2011-12-08| France|           18|
+----------+-------+-------------+

